In [2]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from scipy import stats
from sklearn.metrics import adjusted_rand_score
import pandas as pd
from itertools import product

In [63]:
p_s = [100]
d_s = [2,5,8]
M_s = [4]
n_percluster_s = [50]
conc_dir_s = [0.1,0.5,1]
rho_s = [5, 10, 20]

# Outer cycle for reading the different datasets and perform the estimation
for p,M,npc in product(p_s, M_s, n_percluster_s):
    # for each of the combination of settings I create one dataframe
    
    list_compare = []
    ar_ind = []
    
    for dtrue in d_s:

        for conc_dir in conc_dir_s:

            with open("lamb_out/lamb_p_{0}_d_{1}_M_{2}_npc_{3}_out/conc_{4}_out1/df_performance_lamb.csv".format(p,dtrue,M,npc,conc_dir), newline='') as my_csv:
                lamb_df = pd.read_csv(my_csv)


            list_compare.append(lamb_df[["mode_nclus","avg_nclus","ari_best_clus","CI_aris"]])


        for rho in rho_s:

            with open("applam/app_p_{0}_d_{1}_M_{2}_npc_{3}_out/rho_{4}_out_0/df_performance.csv".format(p,dtrue,M,npc,rho), newline='') as my_csv:
                applam_df = pd.read_csv(my_csv)

            list_compare.append(applam_df[["mode_nclus","avg_nclus","ari_best_clus","CI_aris"]])


        df_final = pd.concat(list_compare)

        ar_lamb = [np.repeat(dtrue,3),np.repeat("Lamb",3),conc_dir_s]
        ar_lamb = list(zip(*ar_lamb))

        ar_app = [np.repeat(dtrue,3),np.repeat("APPLAM",3),rho_s]
        ar_app = list(zip(*ar_app))

        ar_ind = [*ar_ind, *ar_lamb,*ar_app]
        print(ar_ind)
        
    
    index = pd.MultiIndex.from_tuples(ar_ind, names=["Latent dim", "Model", "Parameter"])
        
    df_final = df_final.set_index(index)

FileNotFoundError: [Errno 2] No such file or directory: 'lamb_out/lamb_p_100_d_2_M_4_npc_50_out/conc_0.1_out1/df_performance_lamb.csv'

In [58]:
df_final

mode_nclus  avg_nclus  ari_best_clus  \
Model  Parameter                                         
Lamb   0.1                 6     5.8510       0.923497   
       0.5                27    27.1990       0.265839   
       1.0                30    29.7915       0.242947   
APPLAM 5.0                 6     6.2750       0.857560   
       10.0                6     6.4100       0.841202   
       20.0                7     6.9220       0.850102   

                                                     CI_aris  
Model  Parameter                                              
Lamb   0.1          (0.9007353409732807, 0.9047855669882504)  
       0.5         (0.25434730072910405, 0.2559705342377453)  
       1.0        (0.23754868916974473, 0.23903275022231504)  
APPLAM 5.0          (0.8219625111671286, 0.8251249077123758)  
       10.0         (0.8366691839253457, 0.8402379514053262)  
       20.0         (0.8366251744908427, 0.8407001467733958)

In [34]:
conc_dir_s

[0.1, 0.5, 1]

In [ ]:
list_performance = list()
        nclus_lamb = np.array([len(np.unique(alloc_matrix[i,:])) for i in range(alloc_matrix.shape[0])])
        post_mode_nclus_lamb = stats.mode(nclus_lamb)[0][0] # store in dataframe
        post_avg_nclus_lamb = nclus_lamb.mean() # store in dataframe

        best_clus_lamb = cluster_estimate(alloc_matrix)
        true_clus = np.repeat(range(M),npc)
        ari_best_clus_lamb = adjusted_rand_score(true_clus, best_clus_lamb) # store in dataframe

        aris_chain_lamb = np.array([adjusted_rand_score(true_clus, alloc_matrix[i,:]) for i in range(alloc_matrix.shape[0])])
        mean_aris_lamb, sigma_aris_lamb = np.mean(aris_chain_lamb), np.std(aris_chain_lamb) # store mean_aris in dataframe
        CI_aris_lamb = stats.norm.interval(0.95, loc=mean_aris_lamb, scale=sigma_aris_lamb/sqrt(len(aris_chain_lamb))) # store in dataframe
        list_performance.append([p,dtrue,M,npc,conc_dir,post_mode_nclus_lamb,
                                    post_avg_nclus_lamb,ari_best_clus_lamb,CI_aris_lamb])

        #list_compare.append(["lamb",conc_dir, post_mode_nclus_lamb, post_avg_nclus_lamb,ari_best_clus_lamb,CI_aris_lamb])

        # Save in proper folders similar quantities than for applam
        df_performance_lamb = pd.DataFrame(list_performance, columns=('p','d','M','npc','dir_concentr','mode_nclus', 'avg_nclus', 'ari_best_clus', 'CI_aris'))
        df_performance_lamb.to_csv("data/Student_data/lamb_out/lamb_p_{0}_d_{1}_M_{2}_npc_{3}_out/conc_{4}_out1/df_performance_lamb.csv".format(p,dtrue,M,npc,conc_dir))

        
  #for rho in rho_s:
    
     #with open("data/Student_data/applam/app_p_{0}_d_{1}_M_{2}_npc_{3}_out/rho_{0}_out".format(p,dtrue,M,npc,rho), newline='') as my_csv:
       #df_perf_applam = pd.read_csv(my_csv, sep=',')

     #list_compare.append(["applam", df_perf_applam[["intensity",'mode_nclus', 'avg_nclus', 'ari_best_clus', 'CI_aris']])


    #df_compare = pd.DataFrame(list_compare, columns=('Model','dir_concentr/intens','mode_nclus', 'avg_nclus', 'ari_best_clus', 'CI_aris'))

